In [ ]:
## Estimation of hydraulic propeties using moddeling tools

### GRANAR:
Upon a small set of anatomical features the GRANAR model is able to reconstruct a generic root cell network for mono and dicotyledon.

Change kernel to R to launch the following script.

In [ ]:
# Loading R library
library(tidyverse)
library(plyr)
library(deldir)
library(sp)
library(xml2)
library(viridis)

source("./GRANAR/R/granar.R")

In [ ]:
# Load one parameter file for GRANAR
params <- read_param_xml("GRANAR/model_params/Zea_mays_2_Heymans_2019.xml")

 # # # # # # # # # # # # # # #
#  To change paramter value   #
 # # # # # # # # # # # # # # #
    
# Xylem size (diameter)
params$value[params$type == "max_size" & params$name == "xylem"] <- 0.026

# aerenchyma proportion
params$value[params$type == "proportion" & params$name == "aerenchyma"] <- 0.2
# number of lacuna
params$value[params$type == "n_files" & params$name == "aerenchyma"] <- 15

# Generate the anatomy
sim <- create_anatomy(parameters = params, verbatim=F, paraview = F)
# sim is a list
# sim$nodes is a data frame with most of the information
# sim$output is a data frame with summarize anatomical features

In [ ]:
# To visualize the anatomy and the scenario that are going to be tested.
# you can use the plot_anatomy function.
plot_anatomy(sim, col = "segment", apo_bar = 1)
plot_anatomy(sim, col = "segment", apo_bar = 2)
plot_anatomy(sim, col = "segment", apo_bar = 3)

# To visualize cell type:
plot_anatomy(sim) # default type

In [ ]:
# write geometry
write_anatomy_xml(sim, "./MECHA/cellsetdata/current_root.xml")
# give explicit location of root lacuna in geometry
aer_in_geom_xml(sim, path = "./MECHA/Projects/granar/in/Maize_Geometry.xml")

In [ ]:
# hydraulic conductivity of standard walls
kw = 2.4E-4 # [cm^2/hPa/d] Zhu and Steudle (1991)

# Cell membrane permeability, with separate contribution of the biphospholipid layer (km) and AQP (kAQP)
km = 3.0E-5 # [cm/hPa/d]  after removal of kAQP and kpl from Elhert et al. (2009) and Bret-Hart and Silk (1994)
kAQP = 4.3E-4 # [cm/hPa/d] uniformly distributed on each cell layer

# Individual plasmodesma conductance
kpl = 5.3E-12 #  [cm^3/hPa/d/plasmodesmata] Geometrical average from Bret-Hart and Silk (1994)

# change cell hydraulic properties
microhydro(path = "MECHA/Projects/granar/in/Maize_Hydraulics.xml",
           kw, km, kAQP, kpl)